## Astrobites Scheduling Code
by Mitchell Cavanagh, Nov 2021
***
Code to generate schedules for Astrobites. A schedule is defined as a list of blocks, where each block contains tuples of writer-editor pairs, e.g (AB,CD). The code consists of a class named `Schedule()` in the file `astrobites.py`. I've used the word block instead of week since blocks are designed to contain an arbitrary number of posts.

### Requirements
- Python 3.8 or above
- Numpy

### Overview
The code is designed to satisfy the following schedule requirements:
- Each author writes and edits exactly `num_writes` times
- Writers do not edit their own post
- Writers do not write in consecutive blocks
- Editors do not edit in consecutive blocks
- Writers write at least one regular post*


There is code to enforce the following requirement 
- Writers/editors do not edit/write in the previous block


However it is currently disabled (commented out) as this leads to a massive blowout in generation times (I've yet to get it to generate a schedule with this requirement)

*only when `num_writes > num_queue + num_beyond`, otherwise there seems to be problems :(


### Key Parameters
The following are the main parameters that govern how the code runs:
- `num_writes` How many times each writer should write (default=3)
- `num_regular` The number of regular posts (default=5)
- `num_queue` The number of queue posts (default=1)
- `num_beyond` The number of beyond posts (default=1)
- `max_trials` Maximum number of attempts (default=1000)
- `max_iter` Maximum number of iterations within each attempt (default=200000)


### Authors
The code is designed to work with an arbitrary list of authors, and will (by default) look for a list of author initials in the file `authors.txt`. The list should be formatted with author initials separated by newlines, e.g
```
AB
AC
AD
AE
```

### Caveats and Limitations
The code predominantly uses brute-force to randomly pick writers and editors until it generates a queue that satisfies the schedule constraints and requirements. It does this one block at a time from start to finish. In the default case (5 regular posts, 1 queue and beyond post each, 3 writes per writer), it typically requires 50-100 trials to obtain a suitable schedule (roughly 20-30min). Recommended running the script in the background.


The code is designed to be flexible, however it will struggle with large block sizes and/or large values of `num_writes`, `num_regular`. Note that runnings times also depend on the number of authors; the more authors, the easier it is to schedule blocks.


Queue and beyond posts will only be selected for full blocks. Incomplete blocks (typically the last one as it is the remainder block) will not have queue/beyond posts selected.


`write_schedule()` is mostly meant to be used for blocks of length 7 or more. Also, I've not tested with anything set to 0, so do so at your own risk!


As always, if it appears the code is stuck in an infinite loop, just terminate the kernel or Ctrl-C to force quit.

### Contact

For any issues with the code, please reach out at `mitchell.cavanagh@research.uwa.edu.au`

***
## Demo

In [9]:
import numpy as np

import astrobites

Let's start with a simple demo. First instantiate a scheduler

In [10]:
schedule = astrobites.Schedule()

The only class attribute is the list of authors, which by default is read from the file `authors.txt`. First check to see if the authors have been loaded properly:

In [11]:
print(schedule.authors)

['AC', 'ACu', 'AG', 'AM', 'APic', 'APiz', 'AW', 'CC', 'CJ', 'CM', 'DG', 'GD', 'GFA', 'HS', 'IM', 'JH', 'JN', 'JS', 'JSt', 'KG', 'KP', 'LA', 'LL', 'LZag', 'LZal', 'MC', 'MH', 'OC', 'PG', 'PH', 'RG', 'RH', 'SK', 'SN', 'SS', 'SW', 'VK', 'WJG', 'WY', 'ZS']


You can also randomise authors with `randomise_authors`

In [12]:
schedule.randomise_authors(40,3) # 40 random initials of length 3
print(schedule.authors)

{'YDA', 'GXJ', 'HRK', 'AKV', 'VRF', 'KRV', 'CHB', 'HIY', 'ZCF', 'XYM', 'PSU', 'WGP', 'PET', 'XHJ', 'GBO', 'WVL', 'VIR', 'YUN', 'NUA', 'TOM', 'HGC', 'SJW', 'JRK', 'PAT', 'YFK', 'NCE', 'LWT', 'YSK', 'WNK', 'VWY', 'DTF', 'OCU', 'EDT', 'FSA', 'SHP', 'CYT', 'KLJ', 'IAE', 'BFS', 'RCQ'}


This is very useful for quick testing as the code converges much more quickly (in the default settings) with a larger author list.

Let's go for a default schedule by running `make_schedule()`. By default this will write the schedule immediately to the file `schedule.csv`, but we can either disable that or supply a different filepath. By default, verbose mode is on, and this is recommended to track iterations and trials to see if `max_iter` needs changing.

In [ ]:
# default schedule
blocks, queue_posts, beyond_posts = schedule.make_schedule()

Other examples:

In [ ]:
# make a schedule where every author writes twice
blocks, queue_posts, beyond_posts = schedule.make_schedule(num_writes=2) 

# make a schedule but don't write it to file yet...
blocks, queue_posts, beyond_posts = schedule.make_schedule(write_csv=False)
# ...instead explicitly write it using write_schedule...
schedule.write_schedule(blocks, queue_posts, beyond_posts, f_out='myfilepath')
# ...or just exports the blocks in a plain csv file (useful for manually asigning queue/beyond)
schedule.export_blocks(blocks)

Note that `write_schedule` does not format write/editor pairs in the order that they are in the block. Instead, posts are allocated to days in random order.

## Extensions

The code is designed with flexibility in mind, and can easily be adapted to any use-case where there is a need to group people into pairs. A simple example is creating a "secret santa" list, i.e a list where everyone in a group is assigned some random person to buy a present for. A "secret santa" style list can be called either by using `make_schedule()` with some adjustments to the parameters, or calling the built-in `make_secret_santa()` function.

In [ ]:
# secret santa with make_schedule...
blocks, queue_posts, beyond_posts = schedule.make_schedule(num_writes=1,
    num_regular=1, num_queue=0, num_beyond=0, verbose=False, write_csv=False)

# or with the built-in, which writes to secret_santa.csv
schedule.make_secret_santa()